# Benchmarks - Kernels

In MagmaClust and GPs in general, kernels are what makes it possible to compute a covariance matrix for any set of points, given a set of hyperparameters. They are used in many innermost loops, and their implementation is critical for performance.

**Main considerations when implementing kernels**

A good kernel implementation must be:
* fast, as it is used in many innermost loops
* usable at many dimensions, including a batch dimension with distinct hyperparameters for each element
* work on padded inputs (aka inputs with NaNs), maybe using a mask
* jittable, as it is used in many jit-compiled functions
* modular, as kernels can be combined in many ways
* both static and instance-based, to either carry around hyperparameters or be called with them
* easy to override, as users may want to define their own kernels

These goals are conflicting in some cases (e.g: a jittable version of a kernel is not trivial to write for most people), and the best implementation will depend on the specific use case.

---
## Setup

In [1]:
# Standard library
import os
from functools import partial

os.environ['JAX_ENABLE_X64'] = "True"

In [2]:
# Third party
import jax
from jax import numpy as jnp

import numpy as np
import pandas as pd

In [3]:
# Local
from MagmaClustPy.utils import generate_dummy_db, preprocess_db

In [4]:
# Config
key = jax.random.PRNGKey(0)

---
## Data

---
## Current implementation

In [5]:
import Kernax

---
## Custom implementation(s)

*Start by copy-pasting the original function from the MagmaClustPy module, then bring modifications*

### Defaults of the previous implementation that we wish to correct/improve:

**We want to be able to compose kernels**

As kernels are pytrees, they need to support tree_flatten and tree_unflatten methods.
This is not trivial to implement, as e.g. a sum kernel will have to dynamically know how many hyperparameters belong to each sub-kernel.

---
## Comparison

### On padded datasets

In [6]:
db = pd.read_csv("../datasets/small_distinct_input_shared_hp.csv")
padded_inputs, padded_outputs, mappings, all_inputs = preprocess_db(db)
padded_inputs.shape, padded_outputs.shape, mappings.shape, all_inputs.shape

((20, 15, 1), (20, 15, 1), (20, 15), (41, 1))

In [7]:
noises = jnp.arange(1., 2., .05)
noises.shape

(20,)

In [8]:
old_kernel = Kernax.SEMagmaKernel(length_scale=jnp.array(.3), variance=jnp.array(1.))
new_kernel = Kernax.SEMagmaKernel(length_scale=.3, variance=1.) + Kernax.DiagKernel(Kernax.ExpKernel(noises))

In [9]:
new_kernel.right_kernel.inner_kernel.inner_kernel

ConstantKernel(value=[1.   1.05 1.1  1.15 1.2  1.25 1.3  1.35 1.4  1.45 1.5  1.55 1.6  1.65
 1.7  1.75 1.8  1.85 1.9  1.95])

In [10]:
# Covariance on padded matrix
res1 = old_kernel(padded_inputs)
res1.shape

(20, 15, 15)

In [11]:
# Covariance on un-padded matrix
# res2 = new_kernel(padded_inputs)
res2 = new_kernel(padded_inputs)
res2.shape

(20, 15, 15)

In [12]:
# Check that values in un-padded matrix correspond to the values in the padded matrix
jnp.allclose(jnp.nan_to_num(res1), jnp.nan_to_num(res2))

Array(False, dtype=bool)

In [13]:
%timeit old_kernel(padded_inputs.squeeze()).block_until_ready()

40.3 μs ± 4.25 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [14]:
%timeit new_kernel(padded_inputs).block_until_ready()

65.3 μs ± 3.07 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [15]:
%timeit old_kernel(padded_inputs[0].squeeze()).block_until_ready()

55.5 μs ± 534 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [16]:
%timeit new_kernel(padded_inputs[0]).block_until_ready()

TypeError: cond branches must have equal output types but they differ.

true_fun is <lambda> at /Users/simonlejoly/Documents/Work/MagmaClustPy/Kernax/AbstractKernel.py:36
false_fun is <lambda> at /Users/simonlejoly/Documents/Work/MagmaClustPy/Kernax/AbstractKernel.py:37

The output of true_fun has type float64[] but the corresponding output of false_fun has type float64[20], so the shapes do not match.

Revise true_fun and/or false_fun so that all output types match.

In [20]:
%timeit new_kernel.cross_covariance(padded_inputs[0].reshape(-1, 1), padded_inputs[0].reshape(-1, 1)).block_until_ready()

2.41 ms ± 49.3 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### On scalars

In [26]:
key, subkey = jax.random.split(key)
a = jax.random.uniform(subkey, ())
key, subkey = jax.random.split(key)
b = jax.random.uniform(subkey, ())
a, b

(Array(0.67119299, dtype=float64), Array(0.35166634, dtype=float64))

In [27]:
res1 = old_kernel(a, b)
np.asarray(res1)

array(0.56710711)

In [28]:
res2 = new_kernel(a.reshape(-1, 1), b.reshape(-1, 1))
np.asarray(res2)

array(0.56710711)

In [29]:
jnp.allclose(res1, res2)

Array(True, dtype=bool)

In [30]:
%timeit old_kernel(a, b).block_until_ready()

5.78 μs ± 29.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [31]:
%timeit new_kernel(a.reshape(-1, 1), b.reshape(-1, 1)).block_until_ready()

147 μs ± 1.39 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### On an array and a scalar

In [32]:
key, subkey = jax.random.split(key)
a = jax.random.uniform(subkey, (10000,))
key, subkey = jax.random.split(key)
b = jax.random.uniform(subkey, ())
a, b

(Array([0.15974131, 0.14370377, 0.62062701, ..., 0.7647444 , 0.49158483,
        0.34969892], dtype=float64),
 Array(0.96106866, dtype=float64))

In [33]:
res1 = old_kernel(a, b)
np.asarray(res1)

array([0.02823017, 0.02443875, 0.52524432, ..., 0.80724426, 0.29389634,
       0.12536618])

In [34]:
res2 = new_kernel(a, b)
np.asarray(res2)

IndexError: Too many indices: 0-dimensional array indexed with 1 regular index.

In [42]:
jnp.allclose(res1, res2)

Array(False, dtype=bool)

In [43]:
%timeit old_kernel(a, b).block_until_ready()

47.1 ms ± 3.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [44]:
%timeit new_kernel(a, b).block_until_ready()

33.9 ms ± 9.22 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### On two arrays

In [45]:
key, subkey = jax.random.split(key)
a = jax.random.uniform(subkey, (10000,))
key, subkey = jax.random.split(key)
b = jax.random.uniform(subkey, (15000,))
a.shape, b.shape

((10000,), (15000,))

In [46]:
res1 = old_kernel(a, b)
np.asarray(res1).shape

(10000, 15000)

In [47]:
res2 = new_kernel(a, b)
np.asarray(res2).shape

(10000, 15000)

In [48]:
jnp.allclose(res1, res2)

Array(False, dtype=bool)

In [49]:
%timeit old_kernel(a, b).block_until_ready()

1min 10s ± 5.42 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [50]:
%timeit new_kernel(a, b).block_until_ready()

14.5 s ± 4.42 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


### On two batches of arrays with shared HP

In [51]:
key, subkey = jax.random.split(key)
a = jax.random.uniform(subkey, (50, 100))
key, subkey = jax.random.split(key)
b = jax.random.uniform(subkey, (50, 150))
a.shape, b.shape

((50, 100), (50, 150))

In [52]:
# Using shared hyperparameters for all batches
res1 = old_kernel(a, b)
np.asarray(res1).shape

{'inner_kernel': NewSEMagmaKernel(length_scale=0.3, variance=1.0), 'noise': Array(-2.5, dtype=float64, weak_type=True)} {'length_scale': Array(0.3, dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)} {'inner_kernel': NewSEMagmaKernel(length_scale=0.3, variance=1.0), 'noise': Array(-2.5, dtype=float64, weak_type=True), 'length_scale': Array(0.3, dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)}


(50, 100, 150)

In [53]:
# Also using shared hyperparameters for all batches
res2 = new_kernel(a, b)
np.asarray(res2).shape

{'length_scale': Array(1., dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)} {} {'length_scale': Array(1., dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)}


(50, 100, 150)

In [54]:
jnp.allclose(res1, res2)

Array(False, dtype=bool)

In [55]:
%timeit old_kernel(a, b).block_until_ready()

{'inner_kernel': NewSEMagmaKernel(length_scale=0.3, variance=1.0), 'noise': Array(-2.5, dtype=float64, weak_type=True)} {'length_scale': Array(0.3, dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)} {'inner_kernel': NewSEMagmaKernel(length_scale=0.3, variance=1.0), 'noise': Array(-2.5, dtype=float64, weak_type=True), 'length_scale': Array(0.3, dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)}
{'inner_kernel': NewSEMagmaKernel(length_scale=0.3, variance=1.0), 'noise': Array(-2.5, dtype=float64, weak_type=True)} {'length_scale': Array(0.3, dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)} {'inner_kernel': NewSEMagmaKernel(length_scale=0.3, variance=1.0), 'noise': Array(-2.5, dtype=float64, weak_type=True), 'length_scale': Array(0.3, dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)}
{'inner_kernel': NewSEMagmaKernel(length_scale=0.3, variance=1.0),

In [56]:
%timeit new_kernel(a, b).block_until_ready()

{'length_scale': Array(1., dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)} {} {'length_scale': Array(1., dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)}
{'length_scale': Array(1., dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)} {} {'length_scale': Array(1., dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)}
{'length_scale': Array(1., dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)} {} {'length_scale': Array(1., dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)}
{'length_scale': Array(1., dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)} {} {'length_scale': Array(1., dtype=float64, weak_type=True), 'variance': Array(1., dtype=float64, weak_type=True)}
{'length_scale': Array(1., dtype=float64, weak_type=True), 'variance': Array(1., dty

### On two batches of arrays with distinct HP

In [57]:
key, subkey = jax.random.split(key)
a = jax.random.uniform(subkey, (50, 100))  # 5 batches of 10-dimensional data
key, subkey = jax.random.split(key)
b = jax.random.uniform(subkey, (50, 150))  # 5 batches of 15-dimensional data

# Create distinct hyperparameters for each batch
key, subkey = jax.random.split(key)
distinct_length_scales = jax.random.uniform(subkey, (50,)) + 0.5  # Ensuring positive values
key, subkey = jax.random.split(key)
distinct_variances = jax.random.uniform(subkey, (50,)) + 0.5  # Ensuring positive values

distinct_length_scales, distinct_variances

(Array([1.40087618, 0.50347402, 0.81557193, 1.34923058, 1.00457197,
        0.65788761, 0.9219105 , 0.72545942, 1.23310028, 0.91450771,
        0.52153169, 1.07353706, 0.84001156, 0.63915561, 1.28793987,
        0.960267  , 0.5185922 , 1.46299715, 1.2229984 , 0.73096194,
        1.47447511, 1.41269148, 0.62063165, 1.39239143, 0.56357474,
        0.95879739, 1.23934137, 0.66682831, 1.27697895, 1.41619929,
        1.1099631 , 1.14399616, 1.30613247, 0.78241536, 1.13752401,
        0.7997413 , 0.86489189, 1.1392135 , 1.05768611, 0.64485643,
        0.78303156, 0.61316633, 0.53466457, 0.93155447, 0.81706077,
        0.89970457, 0.6447488 , 1.16024019, 1.30514671, 0.9256145 ],      dtype=float64),
 Array([1.34874564, 1.01689555, 1.11981318, 0.96395389, 0.73946461,
        0.57796114, 0.86174911, 0.87934257, 1.08225335, 1.42074674,
        1.38278328, 0.69312996, 0.62528442, 0.52303255, 1.35397314,
        0.62574799, 1.25923923, 0.82557538, 1.32550512, 1.13693679,
        0.61045989, 0.9583

In [58]:
res1 = old_kernel(a, b, length_scale=distinct_length_scales, variance=distinct_variances)

{'length_scale': Array([1.40087618, 0.50347402, 0.81557193, 1.34923058, 1.00457197,
       0.65788761, 0.9219105 , 0.72545942, 1.23310028, 0.91450771,
       0.52153169, 1.07353706, 0.84001156, 0.63915561, 1.28793987,
       0.960267  , 0.5185922 , 1.46299715, 1.2229984 , 0.73096194,
       1.47447511, 1.41269148, 0.62063165, 1.39239143, 0.56357474,
       0.95879739, 1.23934137, 0.66682831, 1.27697895, 1.41619929,
       1.1099631 , 1.14399616, 1.30613247, 0.78241536, 1.13752401,
       0.7997413 , 0.86489189, 1.1392135 , 1.05768611, 0.64485643,
       0.78303156, 0.61316633, 0.53466457, 0.93155447, 0.81706077,
       0.89970457, 0.6447488 , 1.16024019, 1.30514671, 0.9256145 ],      dtype=float64), 'variance': Array([1.34874564, 1.01689555, 1.11981318, 0.96395389, 0.73946461,
       0.57796114, 0.86174911, 0.87934257, 1.08225335, 1.42074674,
       1.38278328, 0.69312996, 0.62528442, 0.52303255, 1.35397314,
       0.62574799, 1.25923923, 0.82557538, 1.32550512, 1.13693679,
       0.61

In [59]:
res2 = new_kernel(a, b, length_scale=distinct_length_scales, variance=distinct_variances)

{'length_scale': Array([1.40087618, 0.50347402, 0.81557193, 1.34923058, 1.00457197,
       0.65788761, 0.9219105 , 0.72545942, 1.23310028, 0.91450771,
       0.52153169, 1.07353706, 0.84001156, 0.63915561, 1.28793987,
       0.960267  , 0.5185922 , 1.46299715, 1.2229984 , 0.73096194,
       1.47447511, 1.41269148, 0.62063165, 1.39239143, 0.56357474,
       0.95879739, 1.23934137, 0.66682831, 1.27697895, 1.41619929,
       1.1099631 , 1.14399616, 1.30613247, 0.78241536, 1.13752401,
       0.7997413 , 0.86489189, 1.1392135 , 1.05768611, 0.64485643,
       0.78303156, 0.61316633, 0.53466457, 0.93155447, 0.81706077,
       0.89970457, 0.6447488 , 1.16024019, 1.30514671, 0.9256145 ],      dtype=float64), 'variance': Array([1.34874564, 1.01689555, 1.11981318, 0.96395389, 0.73946461,
       0.57796114, 0.86174911, 0.87934257, 1.08225335, 1.42074674,
       1.38278328, 0.69312996, 0.62528442, 0.52303255, 1.35397314,
       0.62574799, 1.25923923, 0.82557538, 1.32550512, 1.13693679,
       0.61

In [60]:
jnp.allclose(res1, res2)

Array(False, dtype=bool)

In [61]:
%timeit old_kernel(a, b, length_scale=distinct_length_scales, variance=distinct_variances).block_until_ready()

{'length_scale': Array([1.40087618, 0.50347402, 0.81557193, 1.34923058, 1.00457197,
       0.65788761, 0.9219105 , 0.72545942, 1.23310028, 0.91450771,
       0.52153169, 1.07353706, 0.84001156, 0.63915561, 1.28793987,
       0.960267  , 0.5185922 , 1.46299715, 1.2229984 , 0.73096194,
       1.47447511, 1.41269148, 0.62063165, 1.39239143, 0.56357474,
       0.95879739, 1.23934137, 0.66682831, 1.27697895, 1.41619929,
       1.1099631 , 1.14399616, 1.30613247, 0.78241536, 1.13752401,
       0.7997413 , 0.86489189, 1.1392135 , 1.05768611, 0.64485643,
       0.78303156, 0.61316633, 0.53466457, 0.93155447, 0.81706077,
       0.89970457, 0.6447488 , 1.16024019, 1.30514671, 0.9256145 ],      dtype=float64), 'variance': Array([1.34874564, 1.01689555, 1.11981318, 0.96395389, 0.73946461,
       0.57796114, 0.86174911, 0.87934257, 1.08225335, 1.42074674,
       1.38278328, 0.69312996, 0.62528442, 0.52303255, 1.35397314,
       0.62574799, 1.25923923, 0.82557538, 1.32550512, 1.13693679,
       0.61

In [62]:
%timeit new_kernel(a, b, length_scale=distinct_length_scales, variance=distinct_variances).block_until_ready()

{'length_scale': Array([1.40087618, 0.50347402, 0.81557193, 1.34923058, 1.00457197,
       0.65788761, 0.9219105 , 0.72545942, 1.23310028, 0.91450771,
       0.52153169, 1.07353706, 0.84001156, 0.63915561, 1.28793987,
       0.960267  , 0.5185922 , 1.46299715, 1.2229984 , 0.73096194,
       1.47447511, 1.41269148, 0.62063165, 1.39239143, 0.56357474,
       0.95879739, 1.23934137, 0.66682831, 1.27697895, 1.41619929,
       1.1099631 , 1.14399616, 1.30613247, 0.78241536, 1.13752401,
       0.7997413 , 0.86489189, 1.1392135 , 1.05768611, 0.64485643,
       0.78303156, 0.61316633, 0.53466457, 0.93155447, 0.81706077,
       0.89970457, 0.6447488 , 1.16024019, 1.30514671, 0.9256145 ],      dtype=float64), 'variance': Array([1.34874564, 1.01689555, 1.11981318, 0.96395389, 0.73946461,
       0.57796114, 0.86174911, 0.87934257, 1.08225335, 1.42074674,
       1.38278328, 0.69312996, 0.62528442, 0.52303255, 1.35397314,
       0.62574799, 1.25923923, 0.82557538, 1.32550512, 1.13693679,
       0.61

### On two batches of arrays with both shared and distinct HP

---
## Conclusion

---